In [331]:
import pandas as pd
import numpy as np
from numba import jit
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.trading import TradingClient
from alpaca.trading.requests import GetCalendarRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.enums import Adjustment

from typing import List, Tuple, Optional

import matplotlib.pyplot as plt
from collections import defaultdict, deque
import numpy as np
import toml
import os
import math

from datetime import datetime
from datetime import datetime, timezone, timedelta

from tqdm import tqdm

# from Trade import Trade, export_trades_to_csv
from TradePosition import TradePosition, export_trades_to_csv
from TouchArea import TouchArea, TouchAreaCollection

from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


trading_client = TradingClient(API_KEY, API_SECRET)

In [332]:
def is_security_marginable(symbol: str) -> bool:
    try:
        asset = trading_client.get_asset(symbol)
        return asset.marginable
    except Exception as e:
        print(f"Error checking marginability for {symbol}: {e}")
        return False
    
# # testing purposes
# def is_security_marginable(symbol: str) -> bool:
#     marginable_symbols = {'NVDA', 'MSFT', 'AAPL', 'TJX', 'JNJ', 'AMZN', 'TSLA', 'AZN'}
#     return symbol in marginable_symbols

# for symbol in {'NVDA', 'MSFT', 'AAPL', 'TJX', 'JNJ', 'AMZN', 'TSLA', 'AZN'}:
#     print(symbol, is_security_marginable(symbol))

In [333]:
def get_market_hours(dates):
    """
    Parameters:
    dates (list of str): List of dates in 'YYYY-MM-DD' format.

    Returns:
    
    dict: Dictionary with dates as keys and (market_open, market_close) tuples as values.
    """
    start_date = min(dates)
    end_date = max(dates)
    
    calendar_request = GetCalendarRequest(start=start_date, end=end_date)
    calendar = trading_client.get_calendar(calendar_request)
    
    market_hours = {str(day.date): (day.open, day.close) for day in calendar}
    return market_hours

In [334]:
def calculate_dynamic_levels(df:pd.DataFrame, ema_short=9, ema_long=20):
    """
    Calculate VWAP and EMAs for the dataframe.
    
    :param df: DataFrame with 'close', 'high', 'low', and 'volume' columns
    :param ema_short: Period for the short EMA (default 9)
    :param ema_long: Period for the long EMA (default 20)
    :return: DataFrame with additional columns for VWAP and EMAs
    """
    
    assert 'vwap' in df.columns
    
    # Calculate EMAs
    # df[f'EMA{ema_short}'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    # df[f'EMA{ema_long}'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    
    # Calculate a combined central value
    # df['central_value'] = (df['vwap'] + df[f'EMA{ema_short}'] + df[f'EMA{ema_long}']) / 3
    # df['central_value'] = df['vwap']
    # df['central_value'] = df[f'EMA{ema_short}']
    # df['central_value'] = df[f'EMA{ema_long}']
    
    # 1
    # df['central_value'] = df['close'].ewm(span=26, adjust=True).mean()
    
    # 2
    # df['central_value'] = df['close'].ewm(span=26, adjust=False).mean()
    
    # 3
    # halflife = '26min'  # 26 minutes
    # df['central_value'] = df['close'].ewm(
    #     halflife=halflife,
    #     times=df.index.get_level_values('timestamp'),
    #     adjust=True
    # ).mean()

    # #4
    # span = 26 # span of 26 = 9.006468342000588min
    # alpha = 2 / (span + 1)
    # halflife = np.log(2) / np.log(1 / (1 - alpha))
    # halflife_str = f"{halflife}min"

    # df['central_value'] = df['close'].ewm(
    #     halflife=halflife_str,
    #     times=df.index.get_level_values('timestamp'),
    #     adjust=True
    # ).mean()
    
    #45
    span = 26 # span of 26 = 9.006468342000588min
    alpha = 2 / (span + 1)
    halflife = np.log(2) / np.log(1 / (1 - alpha))
    halflife_str = f"{halflife}min"

    df['central_value'] = df['close'].ewm(
        halflife=halflife_str,
        times=df.index.get_level_values('timestamp'),
        adjust=True
    ).mean()
    df['central_value'] = (df['vwap'] + df['central_value']*2) / 3

In [355]:
@jit(nopython=True)
def process_touches(touches, prices, touch_area_lower, touch_area_upper, level, level_lower_bound, level_upper_bound, is_long, min_touches):
    consecutive_touches = np.full(min_touches, -1, dtype=np.int64)
    count = 0
    prev_price = None
    for i in range(len(prices)):
        price = prices[i]
        is_touch = (prev_price is not None and 
                    ((prev_price < level <= price) or (prev_price > level >= price)) or 
                    (price == level))
        
        if level_lower_bound <= price <= level_upper_bound:
            if is_touch:
                consecutive_touches[count] = touches[i]
                count += 1
                if count == min_touches:
                    return consecutive_touches
        else:
            buy_price = touch_area_upper if is_long else touch_area_lower
            if (is_long and price > buy_price) or (not is_long and price < buy_price):
                consecutive_touches[:] = -1
                count = 0
        
        prev_price = price
    return np.empty(0, dtype=np.int64)  # Return empty array instead of empty list

def calculate_touch_area(levels_by_date, is_long, df, symbol, market_hours, min_touches, bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time):
    current_id = 0
    touch_areas = []
    widths = []

    for date, levels in tqdm(levels_by_date.items()):
        market_open, market_close = market_hours.get(str(date), (None, None))
        if market_open and market_close and sell_time:
            market_close = min(
                datetime.strptime(str(date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                market_close - timedelta(minutes=3)
            )
        
        if not market_open or not market_close:
            # print('Hours not available. Skipping',date)
            continue
        
        day_data = df[df.index.get_level_values('timestamp').date == date]
        day_timestamps = day_data.index.get_level_values('timestamp')
        day_timestamps_np = day_timestamps.astype(np.int64)  # Convert to nanoseconds
        day_prices = day_data['close'].values
        day_atr = day_data['MTR' if use_median else 'ATR'].values

        for (level_lower_bound, level_upper_bound, level), touches in levels.items():
            if len(touches) < min_touches:
                continue
            
            touch_timestamps_np = np.array([t.timestamp() * 1e9 for t in touches], dtype=np.int64)  # Convert to nanoseconds
            touch_indices = np.searchsorted(day_timestamps_np, touch_timestamps_np)
            
            touch_area_width = touch_area_width_agg(day_atr[touch_indices]) * multiplier

            if touch_area_width is None or np.isnan(touch_area_width) or touch_area_width <= 0:
                continue
            
            widths.append(touch_area_width)
            
            # SUBJECt TO CHANGE
            touch_area_lower = level - (2 * touch_area_width / 3) if is_long else level - (1 * touch_area_width / 3)
            touch_area_upper = level + (1 * touch_area_width / 3) if is_long else level + (2 * touch_area_width / 3)
            
            # touch_area_lower = level - (1 * touch_area_width / 3) if is_long else level - (2 * touch_area_width / 3)
            # touch_area_upper = level + (2 * touch_area_width / 3) if is_long else level + (1 * touch_area_width / 3)
            
            # touch_area_lower = level - (1 * touch_area_width / 2) if is_long else level - (1 * touch_area_width / 2)
            # touch_area_upper = level + (1 * touch_area_width / 2) if is_long else level + (1 * touch_area_width / 2)
            
            
            
            valid_mask = (day_timestamps[touch_indices] >= market_open) & (day_timestamps[touch_indices] < market_close)
            valid_touch_indices = touch_indices[valid_mask]
            valid_prices = day_prices[valid_touch_indices]
            
            consecutive_touch_indices = process_touches(
                valid_touch_indices, 
                valid_prices,
                touch_area_lower, 
                touch_area_upper,  
                level, 
                level_lower_bound,
                level_upper_bound, 
                is_long, 
                min_touches
            )
            
            if len(consecutive_touch_indices) == min_touches:
                consecutive_touches = day_timestamps[consecutive_touch_indices[consecutive_touch_indices != -1]]
                touch_area = TouchArea(
                    id=current_id,
                    level=level,
                    upper_bound=touch_area_upper,
                    lower_bound=touch_area_lower,
                    touches=consecutive_touches.tolist(),
                    is_long=is_long,
                    min_touches=min_touches,
                    bid_buffer_pct=bid_buffer_pct
                )
                touch_areas.append(touch_area)
                current_id += 1

    return touch_areas, widths

In [356]:
def calculate_touch_detection_area(symbol, start_date, end_date, atr_period=10, multiplier=2, min_touches=2, bid_buffer_pct=0.005, sell_time=None, use_median=False, touch_area_width_agg=np.median):
    """
    Calculate the touch detection area based on custom volatility analysis and identify strong support and resistance levels,
    considering only market hours and selling all positions before a specified time.

    Parameters:
    symbol (str|list): The stock symbol to analyze.
    start_date (str|datetime): The start datetime for historical data (format: 'YYYY-MM-DD HH:MM:SS.%f').
    end_date (str|datetime): The end datetime for historical data (format: 'YYYY-MM-DD HH:MM:SS.%f').
    atr_period (int): The period for calculating Average True Range (ATR).
    multiplier (float): The multiplier for ATR to define the touch detection area width.
    min_touches (int): The minimum number of touches to consider a level as strong support or resistance.
    bid_buffer_pct (float): The percentage above the high or below the low to place a stop market buy.
    sell_time (str): The time to sell all positions before (format: 'HH:MM'). If None or after market close, defaults to market close.
    use_median (bool): Flag to switch between using mean or median for levels and touch areas.
    touch_area_width_agg (function): Aggregation function for calculating touch_area_width based on ATR/MTR, which is based on use_median. Suggested: np.median (default), np.nanmax, np.mean
    
    Returns:
    dict: A dictionary with support and resistance areas for both long and short positions.
    """
    assert end_date > start_date
    
    
    # Alpaca API setup
    client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

    # Request historical data
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol, # CAN be str or list. but set up for str (1 stock) for now)
        timeframe=TimeFrame.Minute,
        start=start_date if isinstance(start_date, datetime) else datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S.%f'),
        end=end_date if isinstance(end_date, datetime) else datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S.%f'),
        adjustment=Adjustment.ALL
    )
    bars = client.get_stock_bars(request_params)
    df = bars.df
    
    df.index = df.index.set_levels(pd.Series(df.index.get_level_values('timestamp').to_list()).map(lambda x: x.replace(tzinfo=None)), level='timestamp')
    df.sort_index(inplace=True)
    timestamps = df.index.get_level_values('timestamp')
    print(df.columns)
    # print(df)    
    
    df.reset_index().to_csv('bars.csv',index=False)

    # calculate_dynamic_levels(df, ema_short=9, ema_long=30) # default
    calculate_dynamic_levels(df)

    
    # Calculate True Range (TR)
    df['H-L'] = df['high'] - df['low']
    df['H-PC'] = np.abs(df['high'] - df['close'].shift(1))
    df['L-PC'] = np.abs(df['low'] - df['close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=atr_period).mean()
    df['MTR'] = df['TR'].rolling(window=atr_period).median()
    
    
    # Group data by date
    grouped = df.groupby(timestamps.date)
    
    all_support_levels = defaultdict(dict)
    all_resistance_levels = defaultdict(dict)
    
    
    # def classify_level(level_items, index, day_df):
    #     return 'resistance' if touch_area_width_agg(level_items) > touch_area_width_agg(day_df.loc[index, 'central_value']) else 'support'
    
    def classify_level(level_items, index, day_df):
        return 'resistance' if level_items > day_df.loc[index, 'central_value'] else 'support'
    
    for date, day_df in tqdm(grouped):
        day_timestamps = day_df.index.get_level_values('timestamp')
        
        potential_levels = defaultdict(list)
        w_prev = 0
        
        for i in range(len(day_df)):
            high = day_df['high'].iloc[i]
            low = day_df['low'].iloc[i]
            close = day_df['close'].iloc[i]
            timestamp = day_timestamps[i]
            
            w = (high - low) / 2
            if w == 0:
                w = w_prev
            if w == 0:
                continue
            
            w_prev = w
            
            x = close - w
            y = close + w
            
            # Add this point to its own level
            potential_levels[(x, y)].append(timestamp)
            
            # Check if this point falls within any existing levels
            for (level_x, level_y), touches in potential_levels.items():
                if level_x <= close <= level_y and (level_x, level_y) != (x, y):
                    touches.append(timestamp)

        # Filter for strong levels
        strong_levels = {level: touches for level, touches in potential_levels.items() if len(touches) >= min_touches}

        # Classify levels as support or resistance
        for level, touches in strong_levels.items():
            initial_timestamp = touches[0]
            initial_close = day_df.loc[(symbol, initial_timestamp), 'close']
            
            # touch_prices = day_df.loc[(symbol, touches), 'close'].T.to_list()
            # classification = classify_level(touch_prices[0], (symbol, touches[0]), day_df)
            classification = classify_level(day_df.loc[(symbol, initial_timestamp), 'close'].item(), (symbol, initial_timestamp), day_df)
            
            # print(np.array([str(a) for a in touches]))
            
            if classification == 'support':
                all_support_levels[date][(level[0], level[1], initial_close)] = touches
            else:
                all_resistance_levels[date][(level[0], level[1], initial_close)] = touches

    
    unique_dates = pd.unique(timestamps.date)
    date_strs = [str(date) for date in unique_dates]
    market_hours = get_market_hours(date_strs)
    
    if sell_time:
        sell_time = datetime.strptime(sell_time, '%H:%M')



    # long_touch_area = calculate_touch_area(all_resistance_levels, True)
    # short_touch_area = calculate_touch_area(all_support_levels, False)
    # print(len(long_touch_area), len(short_touch_area))
    
    long_touch_area, long_widths = calculate_touch_area(
        all_resistance_levels, True, df, symbol, market_hours, min_touches, 
        bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time
    )
    short_touch_area, short_widths = calculate_touch_area(
        all_support_levels, False, df, symbol, market_hours, min_touches, 
        bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time
    )
    
    widths = long_widths + short_widths
    
    final_mask = pd.Series(False, index=df.index)
    for date in unique_dates:
        market_open, market_close = market_hours.get(str(date), (None, None))
        date = datetime.strptime(str(date), '%Y-%m-%d')
        if market_open is None or market_close is None:
            market_open = market_close = date
        elif sell_time:
            market_close = date + timedelta(hours=sell_time.hour, minutes=sell_time.minute)
        mask = (timestamps >= str(market_open)) & (timestamps <= str(market_close))
        final_mask |= mask

    return {
        'symbol': symbol,
        'long_touch_area': long_touch_area,
        'short_touch_area': short_touch_area,
        'market_hours': market_hours,
        'bars': df,
        'mask': final_mask,
        'bid_buffer_pct': bid_buffer_pct,
        'min_touches': min_touches,
        'sell_time': sell_time,
        'use_median': use_median
    }
    
    
# example usage:
# touch_detection_areas = calculate_touch_detection_area(symbol, start_date, end_date, atr_period=15, multiplier=1.6, min_touches=3, sell_time='16:00', use_median=True, touch_area_width_agg=np.median)
# balance, trades_executed, percent_increase, trades = backtest_strategy(touch_detection_areas)

In [357]:
def plot_touch_detection_areas(touch_detection_areas, zoom_start_date, zoom_end_date, save_path=None):
    symbol = touch_detection_areas['symbol']
    long_touch_area = touch_detection_areas['long_touch_area']
    short_touch_area = touch_detection_areas['short_touch_area']
    market_hours = touch_detection_areas['market_hours']
    df = touch_detection_areas['bars']
    mask = touch_detection_areas['mask']
    bid_buffer_pct = touch_detection_areas['bid_buffer_pct']
    min_touches = touch_detection_areas['min_touches']
    sell_time = touch_detection_areas['sell_time']
    use_median = touch_detection_areas['use_median']

        
    plt.figure(figsize=(14, 7))
    plt.plot(df.index.get_level_values('timestamp'), df['central_value'], label='central_value', color='yellow')
    plt.plot(df.index.get_level_values('timestamp'), df['close'], label='Close Price', color='blue')

    df = df[mask]
    timestamps = df.index.get_level_values('timestamp')

    # Prepare data structures for combined plotting
    scatter_data = defaultdict(lambda: defaultdict(list))
    fill_between_data = defaultdict(list)
    line_data = defaultdict(list)

    def find_area_end_idx(start_idx, area:TouchArea, market_open, market_close):
        assert market_open.date() == market_close.date()
        entry_price = area.get_buy_price
        
        for i in range(start_idx + 1, len(df)):
            current_time = timestamps[i]
            if current_time >= market_close:
                return i
            current_price = df['close'].iloc[i]
            
            if current_time >= area.get_min_touch_time:
                if area.is_long and current_price >= entry_price:
                    return i
                elif not area.is_long and current_price <= entry_price:
                    return i
            
        return len(df) - 1  # If no end condition is met, return the last index
    
    
    def process_area(area: TouchArea):
        if not area.is_active:
            return
        
        mark_pos = area.get_buy_price
        mark_shape = "v" if area.is_long else '^'
        color = 'red' if area.is_long else 'green'

        
        current_date = None
        for i, touch_time in enumerate(area.touches):
            if touch_time in timestamps:
                start_idx = timestamps.get_loc(str(touch_time))
                
                if timestamps[start_idx].date() != current_date:
                    current_date = timestamps[start_idx].date()
                    market_open, market_close = market_hours.get(str(current_date), (None, None))
                    if market_open and market_close and sell_time:
                        market_close = min(
                            datetime.strptime(str(current_date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                            market_close - timedelta(minutes=3)
                        )
                
                end_idx = find_area_end_idx(start_idx, area, market_open, market_close)
                x1 = [timestamps[start_idx], area.get_min_touch_time]
                x2 = [area.get_min_touch_time, timestamps[end_idx]]
                
                if timestamps[end_idx] >= market_close:
                    continue
                scatter_color = 'gray' if i != min_touches - 1 else 'red' if end_idx == start_idx else 'blue'
                scatter_data[scatter_color][mark_shape].append((touch_time, mark_pos))
                
                if i == 0:  # first touch
                    fill_between_data[color].append((x1 + x2, [area.lower_bound] * 4, [area.upper_bound] * 4))
                    line_data['blue_alpha'].append((x1, [area.level] * 2))
                    line_data['blue'].append((x2, [area.level] * 2))

    for area in tqdm(long_touch_area + short_touch_area):
        process_area(area)

    # Plot combined data
    for color, shape_data in scatter_data.items():
        for shape, points in shape_data.items():
            if points:
                x, y = zip(*points)
                plt.scatter(x, y, color=color, s=12, marker=shape)

    for color, data in fill_between_data.items():
        for x, lower, upper in data:
            plt.fill_between(x[:2], lower[:2], upper[:2], color=color, alpha=0.1)
            plt.fill_between(x[2:], lower[2:], upper[2:], color=color, alpha=0.25)

    for color, data in line_data.items():
        for x, y in data:
            if color == 'blue_alpha':
                plt.plot(x, y, color='blue', linestyle='-', alpha=0.20)
            else:
                plt.plot(x, y, color='blue', linestyle='-')

    plt.title(f'{symbol} Price Chart with Touch Detection Areas')
    plt.xlabel('Date')
    plt.ylabel('Price')
    # plt.legend(['Close Price', 'Long Touch Area', 'Short Touch Area'])
    plt.legend().remove()
    plt.grid(True)
    
    zstart = datetime.strptime(zoom_start_date, '%Y-%m-%d %H:%M:%S.%f')
    zend = datetime.strptime(zoom_end_date, '%Y-%m-%d %H:%M:%S.%f')

    plt.xlim(max(zstart, timestamps[0]), min(zend, timestamps[-1]))

    ymin, ymax = 0, -1
    for i in range(len(timestamps)):
        if timestamps[i] >= zstart:
            print(timestamps[i])
            ymin = i-1
            break
    for i in range(len(timestamps)):
        if timestamps[i] >= zend:
            print(timestamps[i])
            ymax = i
            break
    ys = df['close'].iloc[max(ymin, 0):min(ymax, len(df))]
    plt.ylim(min(ys),max(ys))
    
    if save_path:
        plt.savefig(save_path)

    plt.show()

In [358]:
# Use the decimal module:
# For financial calculations where precision is crucial, consider using Python's decimal module, which provides exact decimal representations and arithmetic.
# https://docs.python.org/3/library/decimal.html

# from decimal import *
# getcontext()
# # Decimal(3.14)

In [359]:
def backtest_strategy(touch_detection_areas, initial_investment=10000, do_longs=True, do_shorts=True, use_margin=False, times_buying_power=4):
    symbol = touch_detection_areas['symbol']
    long_touch_area = touch_detection_areas['long_touch_area']
    short_touch_area = touch_detection_areas['short_touch_area']
    market_hours = touch_detection_areas['market_hours']
    df = touch_detection_areas['bars']
    mask = touch_detection_areas['mask']
    bid_buffer_pct = touch_detection_areas['bid_buffer_pct']
    min_touches = touch_detection_areas['min_touches']
    sell_time = touch_detection_areas['sell_time']
    use_median = touch_detection_areas['use_median']
    
    debug = False
    def debug_print(message):
        if debug:
            print(message)

    POSITION_OPENED = True
    NO_POSITION_OPENED = False
    
    all_touch_areas = []
    assert do_longs or do_shorts
    if do_longs:
        all_touch_areas.extend(long_touch_area)
    if do_shorts:
        all_touch_areas.extend(short_touch_area)
    touch_area_collection = TouchAreaCollection(all_touch_areas, min_touches)
        
    is_marginable = is_security_marginable(symbol)
    if not use_margin or not is_marginable:
        use_margin = False
        times_buying_power = 1
        
        
    print(f'{symbol} is {'NOT ' if not is_marginable else ''}marginable.')
    # return

    trades = []  # List to store all trades
    
    df = df[mask]
    df = df.sort_index(level='timestamp')
    timestamps = df.index.get_level_values('timestamp')

    balance = initial_investment
    total_account_value = initial_investment
    
    trades_executed = 0
    open_positions = {}
    current_id = 0
    

    def update_total_account_value(current_price):
        nonlocal total_account_value, balance
        position_values = sum(position.current_value(current_price) for position in open_positions.values())
        total_account_value = balance + position_values
        
        debug_print(f'  update_total_account_value:')
        debug_print(f'    position_values list: {[f'{position.current_value(current_price):.6f}' for position in open_positions.values()]}')
        debug_print(f'    balance + position_values = {balance:.6f} + {position_values:.6f} = {total_account_value:.6f}')

                        
    def rebalance(add: float = None, set: float = None, current_price: float = None):
        nonlocal balance, total_account_value
        # assert not (add is not None and set is not None)
        if add is not None and set is None:
            debug_print(f'  rebalance:')
            debug_print(f'    balance + add = {balance:.6f} + {add:.6f} = {balance+add:.6f}')
            balance += add
        elif set is not None and add is None:
            balance = set

        # if balance < 0:
        #     print(f'WARNING: Balance < 0: {balance}')
        # # assert balance >= 0
        
        assert balance >= 0

        if current_price is not None:
            update_total_account_value(current_price)
        print(f'Balance: {balance:.4f}, Total Account Value: {total_account_value:.4f}')
                    
    def exit_action(area_id, position):
        nonlocal trades, balance, total_account_value
        
        cash_to_add = position.initial_balance + position.profit_loss
        debug_print(f'  exit_action:')
        debug_print(f'    cash_to_add = position.initial_balance + position.profit_loss = {f'{position.initial_balance:.6f}'} + {position.profit_loss:.6f} = {cash_to_add:.6f}')
        debug_print(f'    open_positions.values() = {[f'{position.current_value(position.exit_price):.6f}' for position in open_positions.values()]}')
                   
        debug_print('  rebalance(add= cash_to_add, current_price=position.exit_price) ')
        rebalance(add=cash_to_add, current_price=position.exit_price)  

        print(f"{'res' if position.is_long else 'sup'} area {area_id}:\t{position.id} {position.exit_time} - Exit {'Long ' if position.is_long else 'Short'} at {position.exit_price:.4f} "
            f"({'+'if position.price_diff >= 0 else ''}{position.price_diff:.4f}). "
            f"Net {'+'if position.profit_loss >= 0 else ''}{position.profit_loss:.4f} "
            f"(ROE: {position.return_on_equity:.2f}%, After -{position.total_transaction_costs:.4f} ({position.entry_transaction_costs:.4f} {position.exit_transaction_costs:.4f} {position.stock_borrow_cost:.4f}) fees)") 
        trades.append(position)

    def close_all_positions(timestamp, exit_price):
        nonlocal trades_executed
        if not open_positions:
            return
        
        positions_to_remove = []

        for area_id, position in list(open_positions.items()):
            position.close(timestamp, exit_price)
            trades_executed += 1
            position.area.record_entry_exit(position.entry_time, position.entry_price, 
                                            timestamp, exit_price)
            position.area.terminate(touch_area_collection)
            positions_to_remove.append(area_id)

        temp = {}
        for area_id in positions_to_remove:
            temp[area_id] = open_positions[area_id]
            del open_positions[area_id]
        for area_id in positions_to_remove:
            exit_action(area_id, temp[area_id])

        assert not open_positions
        open_positions.clear()

    def calculate_max_position_size():
        nonlocal total_account_value
        
        if use_margin and is_marginable:
            initial_margin_requirement = 0.5  # 50% for marginable securities
            overall_margin_multiplier = min(times_buying_power, 4)
            actual_margin_multiplier = min(overall_margin_multiplier, 2.0)
        else:
            initial_margin_requirement = 1.0  # 100% for non-marginable securities
            overall_margin_multiplier = 1.0
            actual_margin_multiplier = 1.0

        available_margin = balance * overall_margin_multiplier
        used_margin = sum(p.initial_balance for p in open_positions.values())
        available_buying_power = available_margin - used_margin
        
        max_position_size = available_buying_power / initial_margin_requirement

        return max_position_size, actual_margin_multiplier, overall_margin_multiplier, initial_margin_requirement

    def place_stop_market_buy(area: TouchArea, timestamp: datetime, open_price: float, high_price: float, low_price: float, close_price: float, prev_close: float):
        nonlocal balance, current_id, total_account_value, open_positions, trades_executed

        if open_positions or balance <= 0:
            return NO_POSITION_OPENED

        debug_print(f"Attempting order: {'Long' if area.is_long else 'Short'} at {area.get_buy_price:.4f}")
        debug_print(f"  Balance: {balance:.2f}, Total Account Value: {total_account_value:.2f}")


        # Check if the stop buy would have executed based on high/low. close price unnecessary (i think)
        if area.is_long:
            if prev_close > area.get_buy_price:
                debug_print(f"  Rejected: Previous close ({prev_close:.4f}) above buy price, likey re-entering area ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
            if high_price < area.get_buy_price:
                debug_print(f"  Rejected: High price ({high_price:.4f}) didn't reach buy price ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
        else:  # short
            if prev_close < area.get_buy_price:
                debug_print(f"  Rejected: Previous close ({prev_close:.4f}) below buy price, likey re-entering area ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
            if low_price > area.get_buy_price:
                debug_print(f"  Rejected: Low price ({low_price:.4f}) didn't reach buy price ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED

        # Stop buy (placed at time of min_touches) would have executed ############### IMPORTANT
        execution_price = area.get_buy_price
        
        # # If not using stop buys and only going by bars
        # if area.is_long:
        #     execution_price = max(close_price, area.get_buy_price)
        # else:
        #     execution_price = min(close_price, area.get_buy_price)

        # Calculate position size, etc...
        max_position_size, actual_margin_multiplier, overall_margin_multiplier, initial_margin_requirement = calculate_max_position_size()

        # Calculate total shares and round down to whole number
        total_shares = math.floor((balance * overall_margin_multiplier) / execution_price)
        
        # # If ensuring even number of stocks:
        # if total_shares % 2 == 1:
        #     total_shares -= 1
            
        # Recalculate invest_amount based on rounded total_shares
        invest_amount = total_shares * execution_price
        actual_cash_used = invest_amount / overall_margin_multiplier
        
        if actual_cash_used > balance:
            debug_print(f"  Order rejected: Insufficient balance ({actual_cash_used:.2f} > {balance:.2f})")
            return NO_POSITION_OPENED

        debug_print(f"  Shares: {total_shares}, Invest Amount: {invest_amount:.2f}")
        debug_print(f"  Margin Multiplier: {actual_margin_multiplier:.2f}")
        
        # Create the position
        position = TradePosition(
            id=current_id,
            area=area,
            is_long=area.is_long,
            entry_time=timestamp,
            entry_price=execution_price,
            initial_balance=actual_cash_used,
            total_shares=total_shares,
            use_margin=use_margin,
            is_marginable=is_marginable,
            times_buying_power=times_buying_power,
            current_stop_price=high_price - area.get_range if area.is_long else low_price + area.get_range,
            max_price=high_price if area.is_long else None, # high price occurs after area.get_buy_price
            min_price=low_price if not area.is_long else None, # low price occurs after area.get_buy_price
            actual_margin_multiplier=actual_margin_multiplier,
        )
        current_id += 1
        
        # position.update_stop_price(close_price)

        # # If we need to immediately exit the position if close price is already less than the buy
        # # Initial tests - poor results. probably dont use
        
        # if (area.is_long and close_price <= area.get_buy_price) or (not area.is_long and close_price >= area.get_buy_price):
        #     # Immediate exit
        #     rebalance(add=-actual_cash_used) # For emulation sake
            
        #     position.update_stop_price(close_price)
        #     position.close(timestamp, close_price)
        #     trades_executed += 1
        #     position.area.record_entry_exit(timestamp, execution_price, timestamp, close_price)
            
        #     cash_to_add = position.initial_balance + position.profit_loss
        #     rebalance(add=cash_to_add, current_price=close_price)
            
        #     temp = f"{high_price:.4f} high" if area.is_long else f"{low_price:.4f} low"
            
        #     print(f"{'res' if area.is_long else 'sup'} area {area.id}: {current_id} {timestamp} - Enter and Immediate Exit {'Long ' if area.is_long else 'Short'} at {close_price:.4f} close after {temp} "
        #         f"({'+'if position.price_diff >= 0 else ''}{position.price_diff:.4f}). "
        #         f"Net {'+'if position.profit_loss >= 0 else ''}{position.profit_loss:.4f} "
        #         f"(ROE: {position.return_on_equity:.2f}%, After -{position.total_transaction_costs:.4f} fees)")

        #     trades.append(position)
        #     return POSITION_OPENED # need to do POSITION_OPENED (True), otherwise the other touch areas will encounter the same and thus cause a string of losses
        # else:
        
        # Add to open positions
        open_positions[area.id] = position
        
        print(f"{'res' if area.is_long else 'sup'} area {area.id}: {current_id} {timestamp} - Enter {'Long ' if area.is_long else 'Short'} at {execution_price:.4f}. "
            f"Shares: {total_shares}, Amount: ${invest_amount:.2f} (Margin: {actual_margin_multiplier:.2f}x, Overall: {overall_margin_multiplier:.2f}x, Sub-positions: {position.num_sub_positions}) "
            f"Bounds: {area.lower_bound:.4f}, {area.upper_bound:.4f}")
        
        rebalance(add=-actual_cash_used, current_price=close_price)
        return POSITION_OPENED
    
    
    

    def update_positions(timestamp, open_price, close_price, high_price, low_price):
        nonlocal trades_executed
        positions_to_remove = []

        def perform_exit(area_id, position):
            nonlocal trades_executed
            position.close(timestamp, position.current_stop_price)
            trades_executed += 1
            position.area.record_entry_exit(position.entry_time, position.entry_price, 
                                            timestamp, position.current_stop_price)
            position.area.terminate(touch_area_collection)
            positions_to_remove.append(area_id)
        
        
        # Check for exits, trailing stop orders, which execute before bar has finished.
        for area_id, position in open_positions.items():
            old_stop_price = position.current_stop_price
            debug_print(f"{timestamp} - Updating position {position.id} (Old stop price: {old_stop_price:.4f})")
            
            # First, update the stop price based on the open price
            # Check if the position would have been stopped out at the open

            position.update_stop_price(open_price)
            debug_print(f"  Stop price at open: {position.current_stop_price:.4f} ")
            if position.should_exit(open_price):
                debug_print(f"  Trailing Stop - Exited at open: {open_price:.4f}")
                perform_exit(area_id, position)
                continue
            
            # If not stopped out at open, simulate intra-minute price movement
            if position.is_long:
                # For long positions, the stop moves up if high price increases
                position.update_stop_price(high_price)
                debug_print(f"  Stop price at high: {position.current_stop_price:.4f} ")
                # Check if low price hit the stop
                if position.should_exit(high_price):
                    debug_print(f"  Trailing Stop - Exited at high: {high_price:.4f}")
                    perform_exit(area_id, position)
                    continue
            else:
                # For short positions, the stop moves down if low price decreases
                position.update_stop_price(low_price)
                debug_print(f"  Stop price at low: {position.current_stop_price:.4f} ")
                # Check if high price hit the stop
                if position.should_exit(low_price):
                    debug_print(f"  Trailing Stop - Exited at low: {low_price:.4f}")
                    perform_exit(area_id, position)
                    continue

            position.update_stop_price(close_price)
            debug_print(f"  Stop price at close: {position.current_stop_price:.4f} ")
            if position.should_exit(close_price):
                debug_print(f"  Trailing Stop - Exited at close: {close_price:.4f}")
                perform_exit(area_id, position)
                    
        temp = {}
        for area_id in positions_to_remove:
            temp[area_id] = open_positions[area_id]
            del open_positions[area_id]
        for area_id in positions_to_remove:
            exit_action(area_id, temp[area_id])
        
        # update_total_account_value(close_price)
        if positions_to_remove:
            debug_print(f"  Updated Total Account Value: {total_account_value:.2f}")

    # print(f"START\nStrategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}.")
    debug_print(f"Strategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}")
    print(f"{timestamps[0]} -> {timestamps[-1]}")
    # print('Initial Investment:', initial_investment)
    debug_print(f"Initial Investment: {initial_investment}, Times Buying Power: {times_buying_power}")
    print('Number of touch areas:', len(all_touch_areas))
    

    daily_data = None
    current_date, market_open, market_close = None, None, None
    for i in tqdm(range(1, len(timestamps))):
        current_time = timestamps[i]
        # print(current_time)
        # print(current_time, len(open_positions))
        
        if current_time.date() != current_date:
            debug_print(f"\nNew trading day: {current_time.date()}")
            # New day, reset daily data
            current_date = current_time.date()
            market_open, market_close = market_hours.get(str(current_date), (None, None))
            if market_open and market_close:
                if sell_time:
                    market_close = min(
                        datetime.strptime(str(current_date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                        market_close - timedelta(minutes=3)
                    )
            daily_data = df[timestamps.date == current_date]
            daily_index = 1 # start at 2nd position
            assert not open_positions
        
        if not market_open or not market_close:
            continue

        if market_open <= current_time < market_close and i != len(df)-1:
            # debug_print(f"\n{current_time} - Market Open")
            
            prev_close = daily_data['close'].iloc[daily_index - 1]
            open_price = daily_data['open'].iloc[daily_index]
            high_price = daily_data['high'].iloc[daily_index]
            low_price = daily_data['low'].iloc[daily_index]
            close_price = daily_data['close'].iloc[daily_index]

            
            update_positions(current_time, open_price, close_price, high_price, low_price)
            
            active_areas = touch_area_collection.get_active_areas(current_time)
            for area in active_areas:
                if balance <= 0:
                    break
                if open_positions:
                    break
                
                if (area.is_long and do_longs) or (not area.is_long and do_shorts):

                    if place_stop_market_buy(area, current_time, open_price, high_price, low_price, close_price, prev_close):
                        break  # Exit the loop after placing a position
            
            daily_index += 1
        elif current_time >= market_close or i >= len(df)-1:
            debug_print(f"\n{current_time} - Market Close")
            close_all_positions(current_time, df['close'].iloc[i])

        # print(len(open_positions))
    if current_time >= market_close:
        assert not open_positions

    percent_increase = ((balance - initial_investment) / initial_investment) * 100

    # Buy and hold strategy
    start_price = df['close'].iloc[0]
    end_price = df['close'].iloc[-1]
    buy_and_hold_increase = ((end_price - start_price) / start_price) * 100

    # export_trades_to_csv(trades,'trades_output.csv')
    
    total_borrow_fees = sum(trade.stock_borrow_cost for trade in trades)
    
    total_profit_loss = sum(trade.profit_loss for trade in trades)
    total_transaction_costs = sum(trade.total_transaction_costs for trade in trades)
    
    total_entry_transaction_costs = sum(trade.entry_transaction_costs for trade in trades)
    total_exit_transaction_costs = sum(trade.exit_transaction_costs for trade in trades)
    total_stock_borrow_costs = sum(trade.stock_borrow_cost for trade in trades)
    
    average_profit_loss = total_profit_loss / len(trades) if trades else 0
    average_profit_loss_percentage = sum(trade.profit_loss_percentage for trade in trades) / len(trades) if trades else 0
    winning_trades = sum(1 for trade in trades if trade.profit_loss > 0)
    losing_trades = sum(1 for trade in trades if trade.profit_loss < 0)
    avg_sub_pos = np.mean([trade.num_sub_positions for trade in trades])

    assert trades_executed == len(trades)
    
    debug_print("\nBacktest Complete")
    debug_print(f"Final Balance: {balance:.2f}")
    debug_print(f"Total Trades Executed: {trades_executed}")
    debug_print(f"Win Rate: {winning_trades / len(trades) * 100:.2f}%")

    # Print statistics
    print(f"END\nStrategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}")
    print(f"{timestamps[0]} -> {timestamps[-1]}")

    print("\nOverall Statistics:")
    print('Initial Investment:', initial_investment)
    print('Final Balance:', balance)
    print(f"Percent increase: {percent_increase:.4f}% ***")
    print(f"If held baseline: {buy_and_hold_increase:.4f}%")
    print('Number of Trades Executed:', trades_executed)
    print(f"\nTotal Profit/Loss (including fees): ${total_profit_loss:.2f}")
    print(f"Total Transaction Costs: ${total_transaction_costs:.2f}")
    print(f"\nBorrow Fees: ${total_borrow_fees:.2f}")
    print(f"Average Profit/Loss per Trade (including fees): ${average_profit_loss:.2f}")
    print(f"Average Profit/Loss Percentage per Trade (including fees): {average_profit_loss_percentage:.2f}%")
    print(f"Average Return on Equity per Trade: {sum(trade.return_on_equity for trade in trades) / len(trades):.2f}%")
    print(f"Number of Winning Trades: {winning_trades}")
    print(f"Number of Losing Trades: {losing_trades}")
    print(f"Win Rate: {winning_trades / len(trades) * 100:.2f}%" if trades else "Win Rate: N/A")
    print(f"\nMargin Usage:")
    print(f"Margin Enabled: {'Yes' if use_margin else 'No'}")
    print(f"Max Buying Power: {times_buying_power}x")
    print(f"Average Margin Multiplier: {sum(trade.margin_multiplier for trade in trades) / len(trades):.2f}x")
    print(f"Average Sub Positions: {avg_sub_pos:.2f}")
    
    
    # return balance, trades_executed, percent_increase, trades
    # return balance, trades_executed, percent_increase, winning_trades, losing_trades, winning_trades / len(trades) * 100
    return balance, trades_executed, percent_increase, winning_trades / len(trades) * 100, total_entry_transaction_costs, total_exit_transaction_costs, \
        total_stock_borrow_costs, total_transaction_costs, avg_sub_pos

In [366]:
# symbol = "NVDA"
# symbol = "MSFT"
# symbol = "AAPL"
# symbol = "TJX"
# symbol = "JNJ"
symbol = "AMZN"
# symbol = "AZN"
# symbol = "TSLA"

# start_date = "2024-06-03 00:00:00.0"
# end_date =   "2024-06-04 00:00:00.0"

# end_date = "2024-03-30 00:00:00.0"


start_date = "2024-02-01 00:00:00.0"
end_date   = "2024-03-06 00:00:00.0"
# start_date = "2024-02-01 00:00:00.0"
# end_date   = "2024-02-01 12:00:00.0"
# start_date = "2024-02-01 00:00:00.0"
# end_date   = "2024-02-02 00:00:00.0"


# start_date = "2023-07-07 00:00:00.0"
# end_date =   "2024-07-07 00:00:00.0"


# start_date = "2023-07-10 00:00:00.0"
# end_date =   "2024-06-26 00:00:00.0"

touch_detection_areas = calculate_touch_detection_area(symbol, start_date, end_date, atr_period=15, multiplier=1.6, min_touches=3, sell_time='16:00', use_median=True, touch_area_width_agg=np.median)


Index(['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'], dtype='object')


100%|██████████| 28/28 [00:01<00:00, 20.08it/s]


In [367]:

# zoom_start_date = "2024-04-15 00:00:00.0"
# zoom_end_date = "2024-04-15 21:00:00.0"

# i, j = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S.%f'), datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S.%f')

# while i <= j:
#     print(i)
#     zoom_start_date = str(i + timedelta(hours=8))
#     zoom_end_date = str(i + timedelta(hours=20))

#     plot_touch_detection_areas(touch_detection_areas, symbol, zoom_start_date, zoom_end_date)
#     i += timedelta(days=1)

zoom_start_date = start_date
zoom_end_date = end_date
# zoom_start_date = "2024-06-03 14:45:00.0"
# zoom_end_date =   "2024-06-04 16:00:00.0"
# plot_touch_detection_areas(touch_detection_areas, zoom_start_date, zoom_end_date)

In [371]:

# # Usage examples:
# print("Both Long and Short:")
backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=True, times_buying_power=2)

# # print("\nLong only:")
# backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=False, times_buying_power=2)
  
# # print("\nShort only:")
# backtest_strategy( touch_detection_areas, use_margin=True, do_longs=False, do_shorts=True, times_buying_power=2)


AMZN is marginable.
2024-02-01 09:32:00 -> 2024-03-05 16:00:00
Number of touch areas: 3502


  6%|▋         | 400/6200 [00:00<00:01, 3992.30it/s]

sup area 10: 1 2024-02-01 09:43:00 - Enter Short at 156.3733. Shares: 63, Amount: $9851.52 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 156.3733, 156.4533
Balance: 148.4800, Total Account Value: 10001.4700
Balance: 10000.8143, Total Account Value: 10000.8143
sup area 10:	0 2024-02-01 09:53:00 - Exit Short at 156.3600 (+0.0133). Net +0.8143 (ROE: 0.01%, After -0.0257 (0.0100 0.0100 0.0057) fees)
sup area 4: 2 2024-02-01 10:34:00 - Enter Short at 156.5680. Shares: 63, Amount: $9863.78 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 156.5680, 156.6640
Balance: 137.0303, Total Account Value: 10002.5783
Balance: 9996.2723, Total Account Value: 9996.2723
sup area 4:	1 2024-02-01 10:55:00 - Exit Short at 156.6360 (-0.0680). Net -4.5420 (ROE: -0.05%, After -0.2580 (0.0100 0.2360 0.0120) fees)
res area 1: 3 2024-02-01 10:55:00 - Enter Long  at 156.6320. Shares: 63, Amount: $9867.82 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 156.5360, 156.6320
Balance: 128.4

 13%|█▎        | 800/6200 [00:00<00:03, 1581.99it/s]

sup area 72: 79 2024-02-02 13:35:00 - Enter Short at 170.2373. Shares: 63, Amount: $10724.95 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 170.2373, 170.4853
Balance: 6.3773, Total Account Value: 10743.1313
Balance: 10762.1361, Total Account Value: 10762.1361
sup area 72:	78 2024-02-02 13:37:00 - Exit Short at 169.7480 (+0.4893). Net +30.8068 (ROE: 0.29%, After -0.0212 (0.0100 0.0100 0.0012) fees)
res area 137: 80 2024-02-02 13:37:00 - Enter Long  at 170.1513. Shares: 63, Amount: $10719.53 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 169.8473, 170.1513
Balance: 42.6021, Total Account Value: 10737.4821
Balance: 10745.7849, Total Account Value: 10745.7849
res area 137:	79 2024-02-02 13:38:00 - Exit Long  at 169.8960 (-0.2553). Net -16.3511 (ROE: -0.15%, After -0.2651 (0.0100 0.2551 0.0000) fees)
res area 172: 81 2024-02-02 13:38:00 - Enter Long  at 169.8879. Shares: 63, Amount: $10702.94 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 169.5041, 169.8879

...

 98%|█████████▊| 6090/6200 [00:01<00:00, 4027.28it/s]

sup area 1570: 1006 2024-03-01 10:52:00 - Enter Short at 176.5020. Shares: 99, Amount: $17473.70 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 176.5020, 176.6460
Balance: 140.6246, Total Account Value: 17618.4806
Balance: 17611.7188, Total Account Value: 17611.7188
sup area 1570:	1005 2024-03-01 10:54:00 - Exit Short at 176.5240 (-0.0220). Net -2.6038 (ROE: -0.01%, After -0.4258 (0.0118 0.4120 0.0020) fees)
res area 1569: 1007 2024-03-01 11:01:00 - Enter Long  at 176.7027. Shares: 99, Amount: $17493.56 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 176.5747, 176.7027
Balance: 118.1548, Total Account Value: 17614.4248
Balance: 17601.3289, Total Account Value: 17601.3289
res area 1569:	1006 2024-03-01 11:03:00 - Exit Long  at 176.6020 (-0.1007). Net -10.3899 (ROE: -0.06%, After -0.4239 (0.0118 0.4122 0.0000) fees)
res area 1567: 1008 2024-03-01 11:23:00 - Enter Long  at 176.6385. Shares: 99, Amount: $17487.21 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds

100%|██████████| 6200/6200 [00:01<00:00, 3209.75it/s]

Balance: 18431.6757, Total Account Value: 18431.6757
sup area 1684:	1098 2024-03-05 14:27:00 - Exit Short at 176.5440 (+0.1813). Net +18.7637 (ROE: 0.10%, After -0.0950 (0.0124 0.0124 0.0702) fees)
res area 1743: 1100 2024-03-05 14:27:00 - Enter Long  at 176.7213. Shares: 104, Amount: $18379.02 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 176.6573, 176.7213
Balance: 52.6571, Total Account Value: 18434.6571
Balance: 18450.4350, Total Account Value: 18450.4350
res area 1743:	1099 2024-03-05 14:29:00 - Exit Long  at 176.9060 (+0.1847). Net +18.7593 (ROE: 0.10%, After -0.4461 (0.0124 0.4337 0.0000) fees)
sup area 1668: 1101 2024-03-05 14:29:00 - Enter Short at 176.5533. Shares: 104, Amount: $18361.55 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1) Bounds: 176.5533, 176.6333
Balance: 88.8883, Total Account Value: 18421.6617
Balance: 18461.1558, Total Account Value: 18461.1558
sup area 1668:	1100 2024-03-05 14:30:00 - Exit Short at 176.4500 (+0.1033). Net +10.7209 (ROE: 0.06%,

(np.float64(18714.709894948977),
 1112,
 np.float64(87.14709894948977),
 53.23741007194245,
 11.793193,
 np.float64(272.89760588975065),
 np.float64(2.2997128278896595),
 np.float64(286.9905117176396),
 np.float64(1.0))

In [372]:
# balance, trades_executed, percent_increase, winning_trades, losing_trades, winning_trades / len(trades) * 100
# return balance, trades_executed, percent_increase, winning_trades / len(trades) * 100, total_entry_transaction_costs, total_exit_transaction_costs, total_stock_borrow_costs, total_transaction_costs

from IPython.utils import io
results = pd.DataFrame()
for i in list(np.arange(0.1, 4.01, 0.1)):
    with io.capture_output() as captured:
        balance, trades_executed, percent_increase, winrate, entry_costs, exit_costs, borrow_costs, total_costs, avg_sub_pos = backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=True, times_buying_power=i)
        newrow = {'times_buying_power':f'{i:.1f}', 'balance':[f'{balance:.2f}'], 'trades_executed':[trades_executed], 'percent_increase_per_trade':[f'{percent_increase/trades_executed:.6f}'], \
            'winrate':f'{winrate:.6f}', 'entry_costs_per_trade':f'{entry_costs/trades_executed:.6f}', 'exit_costs_per_trade':f'{exit_costs/trades_executed:.6f}', \
                'borrow_costs_per_trade':f'{borrow_costs/trades_executed:.6f}', 'total_costs_per_trade':f'{total_costs/trades_executed:.6f}', 'avg_sub_pos':f'{avg_sub_pos:.2f}'}
        results = pd.concat([results, pd.DataFrame.from_dict(newrow)],ignore_index=True)
results

KeyboardInterrupt: 

In [ ]:
results.to_csv('times_buying_power_test.csv',index=False)

In [ ]:
results.astype(float).describe()

,times_buying_power,balance,trades_executed,percent_increase_per_trade,winrate,entry_costs_per_trade,exit_costs_per_trade,borrow_costs_per_trade,total_costs_per_trade,avg_sub_pos
count,40.000000,40.000000,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,2.050000,10861.749000,50.0,0.172350,53.950000,0.019272,0.401625,0.003412,0.424310,1.748500
std,1.169045,505.218681,0.0,0.101044,0.316228,0.008540,0.232224,0.001977,0.242359,0.760093
min,0.100000,10037.080000,50.0,0.007417,52.000000,0.010000,0.026860,0.000154,0.037013,1.000000
25%,1.075000,10437.897500,50.0,0.087581,54.000000,0.010000,0.204379,0.001760,0.216139,1.000000
50%,2.050000,10852.180000,50.0,0.170436,54.000000,0.019678,0.397958,0.003393,0.421129,1.690000
75%,3.025000,11279.327500,50.0,0.255865,54.000000,0.027315,0.595248,0.005053,0.627484,2.500000
max,4.000000,11722.190000,50.0,0.344438,54.000000,0.032149,0.796419,0.006748,0.835316,2.660000
